In [57]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

from keras.utils import to_categorical
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.svm import SVC

Using TensorFlow backend.


In [98]:
data = pd.read_csv('2bhk_folder/2_BHK_20Aug_dataset.csv',index_col = 0)
data.head()

,title,A_1,A_2,A_3,A_4,A_5,BA_1,BA_2,CB_1,CB_2,AC_1,AC_2,Y
0,no_act/19A_noactivity_7(0),-0.982,-0.818,-0.758,-0.768,-0.774,10.654,9.920,2.211,2.061,10.211,9.284,0
1,no_act/19A_noactivity_7(1),-0.981,-0.823,-0.773,-0.806,-0.803,10.491,9.679,2.146,2.108,9.996,9.023,0
2,no_act/19A_noactivity_7(2),-0.984,-0.836,-0.803,-0.789,-0.774,11.017,9.082,2.281,2.089,10.488,8.545,0
3,no_act/19A_noactivity_7(3),-0.988,-0.816,-0.802,-0.810,-0.817,10.353,9.709,2.427,2.146,10.065,9.007,0
4,no_act/19A_noactivity_7(4),-0.987,-0.815,-0.814,-0.772,-0.809,10.704,9.598,2.361,2.217,10.143,9.033,0


In [99]:

features = ['A_1','A_2','A_3','A_4','A_5','BA_1','BA_2','CB_1','CB_2','AC_1','AC_2']
x = data.loc[:, features].values# Separating out the target
y = data.loc[:,['Y']].values# Standardizing the features
x = StandardScaler().fit_transform(x)


In [101]:
## PCA part implementation...

pca = PCA(n_components=11)
principalComponents = pca.fit_transform(x)

PC_df = pd.DataFrame(data = principalComponents,columns = ['PC_1','PC_2','PC_3','PC_4','PC_5','PC_6','PC_7','PC_8','PC_9','PC_10','PC_11'])
PC_df.head()

var = pca.explained_variance_ratio_
print("The array is the cumulative sum of variances covered by each carrier..")
print(np.cumsum(var))

The array is the cumulative sum of variances covered by each carrier..
[0.31081599 0.52541542 0.69165457 0.77542634 0.85626363 0.91693529
 0.96817751 0.99361296 0.99909364 0.99972627 1.        ]


In [102]:
finalDf = pd.concat([data[['title']],PC_df, data[['Y']]], axis = 1)
finalDf.to_csv('PC_2BHK_20Aug_total.csv')
finalDf.head()

,title,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PC_7,PC_8,PC_9,PC_10,PC_11,Y
0,no_act/19A_noactivity_7(0),6.045000,0.508976,0.376056,-0.703347,0.375853,-0.635544,-0.000700,-0.927334,-0.287884,-0.099043,0.003957,0
1,no_act/19A_noactivity_7(1),5.337955,0.479314,1.461703,0.155112,-0.086329,0.064739,-0.571154,-0.639101,-0.256126,-0.045762,0.008387,0
2,no_act/19A_noactivity_7(2),5.263391,0.629173,1.304750,-0.320681,0.943640,-0.568944,0.313521,-0.592343,0.346782,0.233322,0.188796,0
3,no_act/19A_noactivity_7(3),5.062056,0.272582,2.018153,0.560337,-0.366903,0.217519,-0.087362,-0.362420,-0.298200,-0.125362,0.064701,0
4,no_act/19A_noactivity_7(4),5.355381,0.326371,1.504138,0.156067,-0.510696,-0.556705,0.854461,-0.340405,-0.090091,0.006495,0.020585,0


In [104]:
'''
Kfold implementation for with different BHKs..
'''

data = pd.read_csv('PC_2BHK_20Aug_total.csv')

features = ['PC_1','PC_2','PC_3','PC_4','PC_5','PC_6','PC_7','PC_8','PC_9','PC_10','PC_11']


'''
The cross validation technique !!!!
'''
predictions = []


### k fold implementation ...
all_feat = features[0:2] ## 78 percent..

amplitude = features[0:3]#90 per


phase = features[0:7]##96per

features = [all_feat,amplitude,phase]
for feature_list in features:
    print(feature_list)
    #############################################################
    ## k fold validation implementation !!!

    num_folds = 5

    # Define per-fold score containers
    pre_per_fold = []
    acc_per_fold = []


    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=num_folds, shuffle=True)
    inputs =  np.array(data[feature_list])
    targets = np.array(data["Y"])


    # K-fold Cross Validation model evaluation
    fold_no = 1
    for train, test in kfold.split(inputs,targets):

        trainX,trainY = inputs[train],targets[train]
        trainY = trainY.reshape(trainX.shape[0],1)
        testX,testY = inputs[test],targets[test]
        testY = testY.reshape(testX.shape[0],1)


        set_train = np.concatenate((trainX,trainY),axis=1)
        set_test = np.concatenate((testX,testY),axis=1)
        '''
        print('----------------------------------------------------------------------')

        print(f'Training for fold {fold_no} ...')
        '''
        model = SVC(kernel='rbf')
        classifier = model.fit(trainX,trainY)

        y_pred = model.predict(testX) 
        targs = np.array(testY)
        
        targs = np.reshape(targs,(targs.shape[0],))

        preds = y_pred

        for i in range(len(y_pred)):
            if (y_pred[i] == -1):
                y_pred[i] = 0   

        scores = np.array([metrics.accuracy_score(targs, preds),metrics.precision_score(targs, preds)])


    #####################################################################
    #####################################################################
         # Generate a print
        #print(f'Score for fold {fold_no}: acc of {scores[0]}; precision of {scores[1]*100}%')

        pre_per_fold.append(scores[1] * 100)
        acc_per_fold.append(scores[0]*100)



      # Increase fold number
        fold_no = fold_no + 1


    # == Provide average scores ==
    print('------------------------------------------------------------------------')
    print('Score per fold')
    

    '''
        for i in range(0, len(acc_per_fold)):
      print('------------------------------------------------------------------------')
      print(f'> Fold {i+1} - Precison: {pre_per_fold[i]} -  Accuracy : {acc_per_fold[i]}%')
    print('------------------------------------------------------------------------')
    '''

    print('Average scores for all folds:')
    print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
    print(f'> Precision: {np.mean(pre_per_fold)}(+- {np.std(pre_per_fold)}')
    print('------------------------------------------------------------------------')



['PC_1', 'PC_2']


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

------------------------------------------------------------------------
Score per fold
Average scores for all folds:
> Accuracy: 92.70869775392327 (+- 0.5546399735345175)
> Precision: 98.27139465900183(+- 0.9214943178121732
------------------------------------------------------------------------
['PC_1', 'PC_2', 'PC_3']


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

------------------------------------------------------------------------
Score per fold
Average scores for all folds:
> Accuracy: 93.2761722507114 (+- 0.9202348743824107)
> Precision: 98.51921284967743(+- 0.9547571922725864
------------------------------------------------------------------------
['PC_1', 'PC_2', 'PC_3', 'PC_4', 'PC_5', 'PC_6', 'PC_7']


/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/surya/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

------------------------------------------------------------------------
Score per fold
Average scores for all folds:
> Accuracy: 95.5805064262612 (+- 0.6416618523862824)
> Precision: 98.46072373006777(+- 0.5687285162014883
------------------------------------------------------------------------
